This is what will eventually turn into the importable module that does all the backend work for the thing.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_lg')
MAX_LENGTH = 256

2023-04-10 21:13:19.117727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 21:13:22.287734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-10 21:13:22.288007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-10 21:13:22.288037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

In [3]:
def get_tensor(query: str) -> tuple:
    """
    This function returns the tensors for a given text input in three formats 
    (max-pooled, mean-pooled, and padded 256x300), 
    even though we will be discord two of these for now it is stil important 
    because future developments might need us to use one or multiple of the other vectors.
    """
    maxtokens = []
    meantokens = []
    alltokens = []

    doc = nlp(query)
    tokenlist = np.array([token.vector.get() for token in doc])
    maxnormlist = np.array([float(token.vector_norm) for token in doc])
    if MAX_LENGTH - tokenlist.shape[0] > 0:
        constructarr = np.zeros((MAX_LENGTH - tokenlist.shape[0], 300))
        alltokens.append(np.append(tokenlist, constructarr, axis=(0)))
    else:
        alltokens.append(tokenlist[:MAX_LENGTH])

    maxtokens.append(np.array(tokenlist[maxnormlist.argmax()]))
    meantokens.append(np.array(tokenlist.mean(axis=(0))))

    tmaxtokens = tf.convert_to_tensor(maxtokens)
    tmeantokens = tf.convert_to_tensor(meantokens)
    talltokens = tf.convert_to_tensor(alltokens)

    return (tmaxtokens, tmeantokens, talltokens)

In [4]:
gita = pd.read_csv('../dataset/bhagvadgita_encoded.csv')
gita = gita.drop(['Unnamed: 0'], axis=1)
bhagvadgita = gita.drop([546], axis=0) # removing this verse entirely because of very weird artifacts.
quran = pd.read_csv('../dataset/quran_encoded.csv')
quran = quran.drop(['Unnamed: 0'], axis=1)
bible = pd.read_csv('../dataset/bible_encoded.csv')
bible = bible.drop(['Unnamed: 0'], axis=1)

In [6]:
quran.head()

,Name,Surah,Ayat,Verse,Tafseer,a1,a2,a3,a4,a5
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,0.000000,1608.598389,1549.541016,3012.439209,0.0
1,The Opening,1,2,"Praise be to Allah, Lord of the Worlds,",In the Name of God the name of a thing is that...,743.423096,988.568298,1319.458008,0.000000,0.0
2,The Opening,1,3,"The Beneficent, the Merciful.",The Compassionate the Merciful that is to say ...,0.000000,1074.656860,1053.531006,2121.102783,0.0
3,The Opening,1,4,"Owner of the Day of Judgment,",Master of the Day of Judgement that is the day...,0.000000,1818.345703,1718.040283,4036.573486,0.0
4,The Opening,1,5,Thee (alone) we worship; Thee (alone) we ask f...,You alone we worship and You alone we ask for ...,0.000000,0.000000,0.000000,1035.581299,0.0
